In [ ]:
import requests
import pandas as pd

df = pd.read_excel('../data samples/Indications (AMS)_Australia_19.xlsx')
df.head()

In [ ]:
selected_df = df[['Indication', 'Tags','Groups']]
selected_df.head()

In [ ]:
df['Groups'].unique(), df['Groups'].nunique()

In [ ]:
CODE_SYSTEM_ID = '645a4f69203d1d8b3fbb80b4'
# requests create group
group_names = df['Groups'].unique()
group_ress = []
for gname in group_names:
    group_res = requests.post("http://localhost:8000/uil/groups",json={
        "name": gname,
        "code_system_id": CODE_SYSTEM_ID
    }).json()
    if group_res['code']!=200:
        print(gname)
        print(group_res)
        print()
    group_ress.append(group_res)

In [ ]:
for group_res in group_ress:
    print(group_res['data']['id'],group_res['data']['name'])
    

In [ ]:
df_id = pd.DataFrame({
    "GroupId": [res['data']['id'] for res in group_ress],
    "Groups": [res['data']['name'] for res in group_ress]
})
df_id

In [ ]:
groupid_merged_df = pd.merge(selected_df, df_id, left_on='Groups', right_on='Groups', how='left')
groupid_merged_df

In [ ]:
# requests create concept

concept_ress = []
for i, row in groupid_merged_df.iterrows():
    data = {
        'code_system_id':'645a4f69203d1d8b3fbb80b4',
        'group_id':row['GroupId'],
        'name':str(row['Indication']),
        'description':str(row['Tags'])
    }
    print(data)

    concept_res = requests.post("http://localhost:8000/uil/concepts",json=data).json()
    print(concept_res)
    print()
    concept_ress.append(concept_res)